In [178]:
import pandas as pd
import ast
import re

In [179]:
df = pd.read_excel('/home/trishalaswain/Desktop/CHINO_OVERALL_ACCURACY_SHEET.xlsx', index_col=False)

In [180]:
df.shape

(14630, 18)

In [181]:
df.columns

Index(['id', 'physicianId', 'serviceDate', 'processingDate', 'admissionDate',
       'userId', 'lastCodingDate', 'month', 'sheet_accounts', 'EANDM_Final',
       'EANDM_Accepted', 'EANDM_Rejected', 'EANDM_Added', 'EANDM_Suggested',
       'encDocIds', 'local_output_final', 'local_accounts', 'local_output'],
      dtype='object')

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14630 entries, 0 to 14629
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  14630 non-null  int64         
 1   physicianId         14630 non-null  int64         
 2   serviceDate         14630 non-null  datetime64[ns]
 3   processingDate      14630 non-null  datetime64[ns]
 4   admissionDate       14630 non-null  datetime64[ns]
 5   userId              14630 non-null  int64         
 6   lastCodingDate      14630 non-null  datetime64[ns]
 7   month               14630 non-null  object        
 8   sheet_accounts      14630 non-null  object        
 9   EANDM_Final         14625 non-null  object        
 10  EANDM_Accepted      14630 non-null  object        
 11  EANDM_Rejected      14630 non-null  object        
 12  EANDM_Added         14630 non-null  object        
 13  EANDM_Suggested     14630 non-null  object    

In [183]:
df.encDocIds = df.encDocIds.astype('str')

In [184]:
df = df[~df.encDocIds.str.contains(',')]

In [185]:
df.shape

(13701, 18)

In [186]:
new_df = df[~(df.local_output_final=='vlookup_null')]

In [187]:
new_df.shape

(12693, 18)

In [188]:
new_df['lastCodingDate'] = pd.to_datetime(new_df.lastCodingDate)
new_df = new_df[(df.lastCodingDate>='2023-05-01')]
#  & (df.lastCodingDate<='2023-05-01')

/tmp/ipykernel_176697/2386640965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['lastCodingDate'] = pd.to_datetime(new_df.lastCodingDate)
/tmp/ipykernel_176697/2386640965.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = new_df[(df.lastCodingDate>='2023-05-01')]


In [189]:
# new_df['serviceDate'] = pd.to_datetime(new_df.serviceDate)
# new_df = new_df[df.serviceDate>='2023-01-01']

In [190]:
# new_df.local_output_final.value_counts()

In [191]:
# new_df.EANDM_Final.value_counts()

In [192]:
df = new_df.drop(['local_accounts', 'local_output'], axis = 1)

In [193]:
for i,r in df.iterrows():
    df.at[i, 'EANDM_Rejected'] = r['EANDM_Rejected'].replace(']', '').replace('[', '')
    df.at[i, 'EANDM_Added'] = r['EANDM_Added'].replace(']', '').replace('[', '')
    df.at[i, 'EANDM_Accepted'] = r['EANDM_Accepted'].replace(']', '').replace('[', '')
    if (df.at[i, 'EANDM_Rejected'] != '') & (df.at[i, 'EANDM_Accepted'] != ''):
        df.at[i, 'clean_suggested'] = df.at[i, 'EANDM_Accepted'] + ', ' + df.at[i, 'EANDM_Rejected']
    else:
       df.at[i, 'clean_suggested'] = df.at[i, 'EANDM_Accepted']  + df.at[i, 'EANDM_Rejected']
    
    
    if (df.at[i, 'EANDM_Added'] != '') & (df.at[i, 'EANDM_Accepted'] != ''):
        df.at[i, 'clean_final'] = df.at[i, 'EANDM_Accepted'] + ', ' + df.at[i, 'EANDM_Added']
    else:
       df.at[i, 'clean_final'] = df.at[i, 'EANDM_Accepted']  + df.at[i, 'EANDM_Added']

In [194]:
# type(df.at[2,'clean_final'])

In [195]:
# for i,r in df.iterrows():
#     df.loc[i,'EANDM_Suggested_Code'] = ','.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['EANDM_Suggested']))])
#     df.loc[i,'EANDM_Final_Code'] = ','.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['EANDM_Final']))])

In [196]:
# df=df[['processingDate', 'serviceDate', 'encDocIds', 'accountNumber', 'EANDM_Suggested', 'EANDM_Final', 'EANDM_Suggested_Code', 'EANDM_Final_Code']]

In [197]:
# df = df.sort_values(by='serviceDate', ascending=False)

In [198]:
df.clean_final.value_counts()

99284                  984
99285                  242
99291                   70
99283                   68
99223                   50
99284, 99406             8
99292, 99291             6
99223, 99285             6
99285, 99406             4
99292, 99291, 99406      1
99292, 99223             1
99291, 99406             1
99223, 99284             1
Name: clean_final, dtype: int64

In [199]:
for i,r in df.iterrows():
    if str(df.loc[i, 'clean_final']) == '99418, 99223':
        df.loc[i, 'clean_final'] = '99223'
    
    if str(df.loc[i, 'clean_final']) == '99284, 99406':
        df.loc[i, 'clean_final'] = '99284'
        
    if str(df.loc[i, 'clean_final']) == '99284, 99406':
        df.loc[i, 'clean_final'] = '99284'
        
    if str(df.loc[i, 'clean_final']) == '99284, 99406':
        df.loc[i, 'clean_final'] = '99284'
        
    if str(df.loc[i, 'clean_final']) == '99284, 99406':
        df.loc[i, 'clean_final'] = '99284'
        # 
    if str(df.loc[i, 'clean_final']) == '99284, 99406':
        df.loc[i, 'clean_final'] = '99284'
        
    if str(df.loc[i, 'clean_final']) == '99285, 99406':
        df.loc[i, 'clean_final'] = '99285'
        
    if str(df.loc[i, 'clean_final']) == '99223, 99285':
        df.loc[i, 'clean_final'] = '99285'
        
    if str(df.loc[i, 'clean_final']) == '99291, 99406':
        df.loc[i, 'clean_final'] = '99291'
        
    if str(df.loc[i, 'clean_final']) == '99284, 99222':
        df.loc[i, 'clean_final'] = '99284'
        
    if str(df.loc[i, 'clean_final']) == '99283, 99406':
        df.loc[i, 'clean_final'] = '99283'
        
    if str(df.loc[i, 'clean_final']) == '99292, 99223':
        df.loc[i, 'clean_final'] = '99292'
        
    if str(df.loc[i, 'clean_final']) == '99223, 99284':
        df.loc[i, 'clean_final'] = '99284'
        
    if str(df.loc[i, 'clean_final']) == '99292, 99291, 99406':
        df.loc[i, 'clean_final'] = '99292, 99291'
        
    if str(df.loc[i, 'clean_final']) == '99285, 99213':
        df.loc[i, 'clean_final'] = '99285'
        
    if str(df.loc[i, 'clean_final']) == '99223, 99239':
        df.loc[i, 'clean_final'] = '99223'
        
    if str(df.loc[i, 'clean_suggested']) == '99223':
        df.loc[i, 'clean_suggested'] = '99285'
        
    if str(df.loc[i, 'clean_suggested']) == '99222':
        df.loc[i, 'clean_suggested'] = '99284'

    if str(df.loc[i, 'clean_suggested']) == '99221':
        df.loc[i, 'clean_suggested'] = '{No Code}'
        
    if str(df.loc[i, 'clean_suggested']) == '99292, 99291':
        df.loc[i, 'clean_suggested'] = '99291, 99292'

    if str(df.loc[i, 'local_output_final']) == '99223':
        df.loc[i, 'local_output_final'] = '99285'
        
    if str(df.loc[i, 'local_output_final']) == '99222':
        df.loc[i, 'local_output_final'] = '99284'

    if str(df.loc[i, 'local_output_final']) == '99221':
        df.loc[i, 'local_output_final'] = '{No Code}'
        
    if str(df.loc[i, 'local_output_final']) == '99292, 99291':
        df.loc[i, 'local_output_final'] = '99291, 99292'
        
        	
        

In [200]:
df.local_output_final.value_counts()

99284           1018
99285            218
99285             99
99291             55
99291, 99292      24
99282             17
99284              6
99281              4
99283              1
Name: local_output_final, dtype: int64

In [201]:
df.clean_final.value_counts()

99284           993
99285           252
99291            71
99283            68
99223            50
99292, 99291      7
99292             1
Name: clean_final, dtype: int64

In [202]:
# df[df.EANDM_Final_Code.str.contains('99285,99223')]

In [203]:
# df.shape

In [204]:
# df = df[df.lastCodingDate>='2023-04-15']

In [205]:
# df_fac['accepted'] = 0
# df_fac['rejected'] = 0
# df_fac['added'] = 0
# df_fac['precision'] = 0
# df_fac['recall'] = 0
# df_fac['f1_score'] = 0

In [206]:
# df_fac.drop_duplicates(['facility_code'], inplace=True)

In [207]:
# df_fac.reset_index(inplace=True)

In [208]:
# df_fac

In [209]:
df.shape

(1442, 18)

In [210]:
def get_counts(x):
    
    suggested = str(x['local_output_final'])
    final = str(x['clean_final'])
    
    if suggested != '':
        suggested_set = set(suggested.split(', '))
        final_set = set(final.split(', '))
        if '{No Code}' in suggested:
            suggested_set.clear()
            final_set.clear()
        if '9' not in final:
            final_set.clear()
        # if '99221' in suggested:
            # suggested_set.remove('99221')
            # suggested_set.clear()
        
        
    
        accepted = len(suggested_set.intersection(final_set))
        rejected = len(suggested_set - final_set)
        added = len(final_set - suggested_set)
        if (rejected >= 1 or added >= 1):
            print('rejected',rejected)
            print('added',added)
            print('suggested_set',suggested_set,'\t final_set',final_set)
            print('accountNumber', str(x['sheet_accounts']))
            
    # for index, row in df_fac.iterrows():
    #     if (str(row['facility_code']) == str(x['facility_code'])):
    #         old_accepted = df_fac.loc[index,'accepted']
    #         df_fac.at[index,'accepted'] =  old_accepted + accepted
    #         # print('adding', accepted)
    #         old_added = df_fac.loc[index,'added']
    #         df_fac.at[index,'added'] = old_added+ added
    #         old_rejected = df_fac.loc[index,'rejected']
    #         df_fac.at[index,'rejected'] = old_rejected+ rejected

        return pd.Series({'accepted':accepted, 'rejected':rejected, 'added':added})

In [211]:
# df

In [212]:
count_df =  df.apply(get_counts, axis=1)
# def get_accuracy()

rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99283'}
accountNumber V00001043471
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99285'}
accountNumber V00001043498
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99283'}
accountNumber V00001043512
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99283'}
accountNumber V00001043748
rejected 1
added 1
suggested_set {'99285'} 	 final_set {'99223'}
accountNumber V00001044585
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99283'}
accountNumber V00001043751
rejected 1
added 1
suggested_set {'99285'} 	 final_set {'99284'}
accountNumber V00001044462
rejected 1
added 1
suggested_set {'99285'} 	 final_set {'99223'}
accountNumber V00001044438
rejected 1
added 1
suggested_set {'99285'} 	 final_set {'99284'}
accountNumber V00001044448
rejected 1
added 0
suggested_set {'99292', '99291'} 	 final_set {'99291'}
accountNumber V00001044527
rejected 1
added 1
suggested_set {'99285'} 	 final_set {'99284'}
accoun

In [213]:
# for i,r in df.iterrows():
#     get_counts(r,df_fac)

In [214]:
count_df

,accepted,rejected,added
15,0,1,1
17,1,0,0
18,1,0,0
20,1,0,0
21,1,0,0
...,...,...,...
14543,1,0,0
14552,1,0,0
14563,0,1,1
14568,0,1,1


In [215]:
df2 = pd.concat([df, count_df], axis = 1)

In [216]:
df2.reindex()

,id,physicianId,serviceDate,processingDate,admissionDate,userId,lastCodingDate,month,sheet_accounts,EANDM_Final,...,EANDM_Rejected,EANDM_Added,EANDM_Suggested,encDocIds,local_output_final,clean_suggested,clean_final,accepted,rejected,added
15,6750439,19670,2023-04-24,2023-05-02 05:06:48,2023-04-24 11:32:00,4724,2023-05-08 19:58:46,2023-05,V00001043471,99283 = [U7-M/Caid Care Lev 7 State Def],...,99284,99283,{99284=[]},40456954,99284,99284,99283,0,1,1
17,6750437,19670,2023-04-25,2023-05-02 05:06:43,2023-04-25 20:06:00,4724,2023-05-03 04:13:54,2023-05,V00001043714,99284,...,,,{99284=[]},40456948,99284,99284,99284,1,0,0
18,6750435,19670,2023-04-24,2023-05-02 05:06:32,2023-04-24 13:08:00,6248,2023-05-02 17:05:27,2023-05,V00001043487,99284,...,,,{99284=[]},40456935,99284,99284,99284,1,0,0
20,6750447,19670,2023-04-24,2023-05-02 05:07:32,2023-04-24 16:59:00,6248,2023-05-02 17:08:28,2023-05,V00001043524,99284 = [25-Sig/Sep ID E&M Same Pro/Day],...,,,{99284=[]},40456987,99284,99284,99284,1,0,0
21,6750446,19670,2023-04-24,2023-05-02 05:07:24,2023-04-24 11:59:00,4724,2023-05-08 20:36:23,2023-05,V00001043480,99284 = [U7-M/Caid Care Lev 7 State Def],...,,,{99284=[]},40456985,99284,99284,99284,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14543,6913750,19670,2023-05-06,2023-05-09 05:06:48,2023-05-06 16:48:00,6248,2023-05-13 01:34:39,2023-05,V00001045206,99284,...,,,{99284=[]},41446585,99284,99284,99284,1,0,0
14552,6913720,19670,2023-05-06,2023-05-09 05:05:24,2023-05-06 10:02:00,6248,2023-05-13 01:10:58,2023-05,V00001045159,99284,...,,,{99284=[]},41446451,99284,99284,99284,1,0,0
14563,6913719,19670,2023-05-05,2023-05-09 05:05:19,2023-05-05 17:16:00,4724,2023-05-11 16:47:38,2023-05,V00001045088,99284,...,99285,99284,{99285=[]},41446441,99285,99285,99284,0,1,1
14568,6651521,19670,2023-04-26,2023-04-27 08:20:52,2023-04-26 22:26:00,6248,2023-05-03 15:36:19,2023-05,V00001043893,99284,...,99285,99284,{99285=[25]},39899672,99285,99285,99284,0,1,1


In [217]:
count_df.accepted.sum(), count_df.rejected.sum(), count_df.added.sum()

(1208, 258, 241)

In [218]:
precision = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.rejected.sum())
recall = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.added.sum())
f1_score = (2 * precision * recall) / (recall + precision)

In [219]:
data = {"precision":precision, "recall":recall, "f1_score":f1_score}
pd.Series(data).to_frame('new_overall_sheet_stats_local')

,new_overall_sheet_stats_local
precision,0.824011
recall,0.833678
f1_score,0.828816


In [220]:
df.clean_final.value_counts()

99284           993
99285           252
99291            71
99283            68
99223            50
99292, 99291      7
99292             1
Name: clean_final, dtype: int64

In [221]:
df.local_output_final.value_counts()

99284           1018
99285            218
99285             99
99291             55
99291, 99292      24
99282             17
99284              6
99281              4
99283              1
Name: local_output_final, dtype: int64

In [222]:
cs =  pd.crosstab(df.clean_final, df.local_output_final, margins=True)


In [223]:
cs.style.background_gradient(cmap='YlGnBu', vmin=-5, vmax=32)

local_output_final,99281,99282,99283,99284,99285,99291,99284,99285,"99291, 99292",All
clean_final,,,,,,,,,,
99223,0,0,0,0,0,0,4,46,0,50
99283,1,11,1,55,0,0,0,0,0,68
99284,3,6,0,916,63,0,1,4,0,993
99285,0,0,0,47,155,0,1,49,0,252
99291,0,0,0,0,0,55,0,0,16,71
99292,0,0,0,0,0,0,0,0,1,1
"99292, 99291",0,0,0,0,0,0,0,0,7,7
All,4,17,1,1018,218,55,6,99,24,1442


In [224]:
df.clean_suggested = df.clean_suggested.astype('str')
df.clean_final = df.clean_final.astype('str')
df.local_output_final = df.local_output_final.astype('str')

In [225]:
def highlight_same(data):
    
    highlighter = pd.DataFrame("", index=data.index, columns=data.columns)
    for col in data.columns:
        for index in data.index:

 

            if col == index:
                highlighter.loc[index, col] = 'background-color : springgreen'
                
            elif str(col)>str(index) and data.loc[index, col]>10:
                highlighter.loc[index, col] = 'background-color : lightskyblue'

            elif data.loc[index, col]>10:
                highlighter.loc[index, col] = 'background-color : PaleVioletRed'

    return highlighter

 

crosstab = cs.style.apply(highlight_same, axis=None)

In [226]:
crosstab.to_excel('/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/CHINO_ALL_OP/25_MAY/test_matrix.xlsx')

In [227]:
crosstab

local_output_final,99281,99282,99283,99284,99285,99291,99284,99285,"99291, 99292",All
clean_final,,,,,,,,,,
99223,0,0,0,0,0,0,4,46,0,50
99283,1,11,1,55,0,0,0,0,0,68
99284,3,6,0,916,63,0,1,4,0,993
99285,0,0,0,47,155,0,1,49,0,252
99291,0,0,0,0,0,55,0,0,16,71
99292,0,0,0,0,0,0,0,0,1,1
"99292, 99291",0,0,0,0,0,0,0,0,7,7
All,4,17,1,1018,218,55,6,99,24,1442


In [158]:
# confusion_matrix

In [159]:
# cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels = df.clean_suggested.unique().tolist())

In [160]:
# confusion_matrix = pd.crosstab(df.clean_final, df.clean_suggested,margins = True, margins_name='Total')

In [161]:
# confusion_matrix.to_excel('/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/CHINO_ALL_OP/25_MAY/test_matrix.xlsx')

In [162]:
# for i, r in df_fac.iterrows():
    
#     precision = df_fac.at[i,'accepted'] / (df_fac.at[i,'accepted'] + df_fac.at[i,'rejected'])
#     recall = df_fac.at[i,'accepted'] / (df_fac.at[i,'accepted'] + df_fac.at[i,'added'])
#     f1_score = (2 * precision * recall) / (recall + precision)
#     df_fac.at[i,'precision'] = precision
#     df_fac.at[i,'recall'] = recall
#     df_fac.at[i,'f1_score'] = f1_score

In [163]:
# df_fac

In [164]:
# df_fac.to_excel('/home/trishalaswain/Downloads/Script_Testing/excel_1.xlsx')

In [165]:
# precision, recall, f1_score

In [166]:
# def check_suggested(df):
#     for ind,row in df.iterrows():
#         row['new_suggested'] = str(row['new_suggested']).split(',')
#         for item in row['new_suggested']:
#             if str(item) == '{No Code}':
#                 df.at[ind,'rejected'] = 0
#                 continue
#             if str(item) in str(row['final']):
#                 df.at[ind,'accepted'] += 1
#             if str(item) not in str(row['final']):
#                 df.at[ind,'rejected'] +=1

In [167]:
# def check_final(df):
#     for ind,row in df.iterrows():
#         row['final'] = str(row['final']).split(',')
#         for final_code in row['final']:
#             if final_code not in str(row['new_suggested']):
#                 df.at[ind,'added'] +=1

In [168]:
# check_suggested(df)
# check_final(df)

In [169]:
# df['EANDM_Suggested_code'] = None

In [170]:
# def emsugg_filtered(df):
#     df['EANDM_Suggested_code'] = None
#     for ind, row in df.iterrows():

#         li = ast.literal_eval(row["EM_accepted"])
#         li2 = ast.literal_eval(row["EM_rejected"])
#         li3 = li + li2
#         df.at[ind,'EANDM_Suggested_code'] = li3

In [171]:
# emsugg_filtered(df)

In [172]:
# df

In [173]:
# def check_suggested2(df):
#     for ind,row in df.iterrows():
#         for item in row['EANDM_Suggested_code']:
#             if str(item) in str(row['final']):
#                 df.at[ind,'old_accepted'] += 1
#             if str(item) not in str(row['final']):
#                 df.at[ind,'old_rejected'] +=1

In [174]:
# def check_final2(df):
#     for ind,row in df.iterrows():
#         row['final'] = str(row['final']).split(',')
#         for final_code in row['final']:
#             if final_code not in str(row['EANDM_Suggested_code']):
#                 df.at[ind,'old_added'] +=1

In [175]:
# check_suggested2(df)
# check_final2(df)

In [176]:
df2 = pd.read_excel('/home/trishalaswain/Downloads/Script_Testing/CHINO_ALL_TEXT_&_OUTPUT.xlsx', index_col=False)

In [177]:
df2.final_code.value_counts()

99284    7000
99285    2623
99283    1817
99291     713
99223     298
None      144
99282     128
99406     116
99418      55
99292      33
99220      16
99222      16
993X0      11
99236       4
99219       3
99221       3
99281       2
99218       2
99239       1
99215       1
99359       1
Name: final_code, dtype: int64